In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your OpenAI key"

from langchain.llms import OpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader


ABS_PATH = os.path.dirname(os.path.abspath(''))
DB_DIR = os.path.join(ABS_PATH, "db")
FILE_DIR = os.path.join(ABS_PATH, "docs")

embeddings = OpenAIEmbeddings()

llm = OpenAI(temperature=0)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

loader = DirectoryLoader(FILE_DIR, glob="**/*.txt", show_progress=True)
docs = loader.load()
texts = text_splitter.split_documents(docs)

db = Chroma.from_documents(texts, embeddings, collection_name="docstore", persist_directory=DB_DIR)

retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

In [ ]:
#query = input("Enter query:") #interactive Q&A

ans = qa({"query": "your query"})

print(ans)